In [2]:
import os
import glob
import random
import json
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm,trange
import csv
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# import torch
# import torchvision
# from torchvision import transforms
import matplotlib.pyplot as plt
from pandas import DataFrame

In [14]:
def number_of_subfiles(train_dir):
    s = 0
    lists = []
    for (path, dir, files) in os.walk(train_dir):
        for filename in files:
            ext = os.path.splitext(filename)[-1]
            # print(path+"/"+filename)
            if ext != '.zip':

                file_index = os.path.splitext(filename)[0]
                if int(file_index[-6:]) >= 30000 and int(file_index[-6:]) < 60000:
                    s+=1
                    # print(os.path.splitext(filename)[0])
                    lists.append(path+"/"+filename)
    return s, lists

# cnt_image, list_image = number_of_subfiles("/hdd1/image/1. 가로형간판/")
# cnt_label, list_label = number_of_subfiles("/hdd1/label/1.간판/1.가로형간판/가로형간판2/")
cnt_image, list_image = number_of_subfiles("D:\Dacon\야외 실제 촬영 한글 이미지\[원천]Training_간판_가로형간판_원천데이터2")
# D:\Dacon\야외 실제 촬영 한글 이미지\[라벨]Training\1.간판\1.가로형간판\가로형간판2
cnt_label, list_label = number_of_subfiles("D:/Dacon/야외 실제 촬영 한글 이미지/[라벨]Training/1.간판/1.가로형간판/가로형간판2")

print(cnt_image, list_image.__len__())
print(cnt_label, list_label.__len__())
# print(list_image[0])

30000 30000
30000 30000


In [15]:
list_image = sorted(list_image)
list_label = sorted(list_label)


In [16]:
list_image[0], list_label[0]

('D:\\Dacon\\야외 실제 촬영 한글 이미지\\[원천]Training_간판_가로형간판_원천데이터2/간판_가로형간판_030000.jpg',
 'D:/Dacon/야외 실제 촬영 한글 이미지/[라벨]Training/1.간판/1.가로형간판/가로형간판2/간판_가로형간판_030000.json')

In [17]:
json_list = []
for label in tqdm(list_label):    
    with open(label, encoding='utf8') as f:
        json_list.append(json.load(f))

  0%|          | 0/30000 [00:00<?, ?it/s]

In [18]:
json_list[0]

{'annotations': [{'bbox': [435, 366, 937, 431],
   'id': 1,
   'text': 'xxx',
   'image_id': 1},
  {'bbox': [141, 482, 259, 419], 'id': 2, 'text': '늘푸른', 'image_id': 1}],
 'cropLabels': [],
 'images': [{'file_name': '간판_가로형간판_030000.jpg',
   'date_created': '20210126',
   'width': 1600,
   'id': 1,
   'height': 1200}],
 'info': {'date_created': '20210126',
  'name': '간판_가로형간판_030000',
  'description': ''},
 'metadata': [{'area': '경상권',
   'wordfont': '캘리그라피',
   'outline': '상',
   'illuminance': '밝음',
   'wordsize': '중',
   'light': '자연광',
   'wordconnection': '빈공간없이 이어짐',
   'subclass': '가로형간판',
   'weather': '맑음',
   'device': '스마트폰',
   'wordcolor': '단색',
   'class': '실외간판',
   'wordorientation': '세로'}]}

In [19]:
label = []
lotate  = []
idxs = []
# list_image, list_label

# idx -> number, i is info
for idx,i in tqdm(enumerate(json_list),total=len(json_list)):
    for jdx,j in enumerate(i['annotations']):
        # if j['bbox'] == [None, None, None, None]: continue
        # if j['bbox'] == [None, None, None, None]: 
        if j['text'] == "xxx": 
            idxs.append(idx)
            lotate.append(j['bbox'])
            label.append(j['text'])
        # elif j['text'] == "xxx": continue
        # elif j['text'] == " ": continue



  0%|          | 0/30000 [00:00<?, ?it/s]

In [20]:
idxs.__len__(), lotate.__len__(), label.__len__()

(69901, 69901, 69901)

In [21]:
for i in trange(len(lotate)):
    lotate[i][2] = lotate[i][0]+lotate[i][2]
    lotate[i][3] = lotate[i][1]+lotate[i][3]

  0%|          | 0/69901 [00:00<?, ?it/s]

In [23]:
lotate[:5]

[[435, 366, 1372, 797],
 [368, 0, 1152, 299],
 [0, 384, 613, 497],
 [0, 714, 274, 798],
 [1063, 1148, 1347, 1200]]

In [55]:
new_image = []

path = "D:/Dacon/crop_image/"
# alpha = 0.5 #resize할 범위 만약 할거면 값 조정해서 resize해서 저장하면 됨 
start = 0 # default
starts = 11510 # 하다가 오류 떴을 때, 뜬 번호 여기에 넣고 하면 이어서 가능
# bins = 1
for idx,i in tqdm(enumerate(lotate[starts:]),total =len(lotate[starts:])):
    idx+=starts
    image = Image.open(list_image[idxs[idx]])
    crop_image = image.crop(i)

    # crop_image.resize((crop_image.size[0]*alpha,crop_image.size[1]*alpha))#resize하고 싶은 비율 설정 
    crop_image.save(f'{path}/{list_image[idxs[idx]].split(".")[0][-6:]} and {i} train_data_{idx}.jpg')

  0%|          | 0/58390 [00:00<?, ?it/s]

SystemError: tile cannot extend outside image

## 만약 오류가 떴다면

In [59]:
# lotate[11510+69:]

In [60]:
# Image.open(list_image[idxs[idx]]) # 이미지 확인
Image.open(list_image[idxs[11510+69]]).size # 사이즈 확인

(1600, 1200)

In [54]:
## 디버깅 귀찮다면
# 해당위치 = 11510
# lotate.pop(해당위치), idxs.pop(해당위치), label.pop(해당위치),

([1592, 1206, 1600], 4874, 'xxx')

In [56]:
imageScreenshot.crop((x, y, width, height)) # 으로 확인 가능

NameError: name 'imageScreenshot' is not defined